In [6]:
# https://www.kaggle.com/lichtlab/h-m-data-deep-dive-chap-1-understand-article
import pandas as pd
import plotly.express as px
import numpy as np
import seaborn as sns

In [7]:
df_article = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv',dtype={'article_id': str})

def set_gender_flg(x):
    x['is_for_male'] = 0
    x['is_for_female'] = 0
    x['is_for_mama'] = 0
    if x['index_group_name'] in ['Ladieswear','Divided']:
        x['is_for_female'] = 1
    if x['index_group_name'] == 'Menswear':
        x['is_for_male'] = 1
    if x['index_group_name'] in ['Baby/Children','Sport']:
        if 'boy' in x['department_name'].lower() or 'men' in x['department_name'].lower():
            x['is_for_male'] = 1
        if 'girl' in x['department_name'].lower() or 'ladies' in x['department_name'].lower():
            x['is_for_female'] = 1
    if x['section_name'] == 'Mama':
        x['is_for_mama'] = 1
    return x

df_article = df_article.apply(set_gender_flg, axis=1)
df_article.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,is_for_male,is_for_female,is_for_mama
0,0108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0,1,0
1,0108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0,1,0
2,0108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,0,1,0
3,0110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",0,1,0
4,0110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",0,1,0


In [8]:
df_article['idxgrp_idx_prdtyp'] = df_article['index_group_name'] + '_' + df_article['index_name'] + '_' + df_article['product_type_name']

df_trans = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',dtype={'article_id': str})
df_trans = df_trans.sample(frac=0.1) # downsampling due to memory limits
df_trans['t_dat'] = pd.to_datetime(df_trans['t_dat'])
df_trans['YYYY_MM'] = df_trans['t_dat'].dt.year.astype(str) + '_' + df_trans['t_dat'].dt.month.astype(str)
df_trans['year'] = df_trans['t_dat'].dt.year
df_trans['month'] = df_trans['t_dat'].dt.month
df = pd.merge(df_trans, df_article, on='article_id', how='left')
del df_trans, df_article

dfgrp1 = df.groupby(['idxgrp_idx_prdtyp'])[['price']].sum().reset_index()
dfgrp2 = df.groupby(['idxgrp_idx_prdtyp', 'year', 'month'])[['price']].sum().reset_index()
dfgrp2 = pd.merge(dfgrp2, dfgrp1, on='idxgrp_idx_prdtyp', how='left')
dfgrp2['monthsales/ttl-sales'] = dfgrp2['price_x'] / dfgrp2['price_y'] * 100
dfgrp2['ym_date'] = dfgrp2['year'].astype(str) + '-' + dfgrp2['month'].astype(str) + '-1'
dfgrp2['ym_date'] = pd.to_datetime(dfgrp2['ym_date'])


In [9]:
dfgrp2.head()

,idxgrp_idx_prdtyp,year,month,price_x,price_y,monthsales/ttl-sales,ym_date
0,Baby/Children_Baby Sizes 50-98_Accessories set,2020,8,0.022017,0.104441,21.080818,2020-08-01
1,Baby/Children_Baby Sizes 50-98_Accessories set,2020,9,0.082424,0.104441,78.919182,2020-09-01
2,Baby/Children_Baby Sizes 50-98_Baby Bib,2019,6,0.044034,0.467153,9.426021,2019-06-01
3,Baby/Children_Baby Sizes 50-98_Baby Bib,2019,10,0.044034,0.467153,9.426021,2019-10-01
4,Baby/Children_Baby Sizes 50-98_Baby Bib,2019,12,0.044034,0.467153,9.426021,2019-12-01


In [13]:

dfgrp2 = pd.pivot_table(dfgrp2, index='ym_date', columns='idxgrp_idx_prdtyp', 
                        values='monthsales/ttl-sales').reset_index().fillna(0)
display(dfgrp2.head())

fig = px.line(dfgrp2, x='ym_date', y=['Ladieswear_Ladieswear_Jacket',
                                        'Divided_Divided_Jacket',
                                        'Menswear_Menswear_Jacket',
                                        'Ladieswear_Ladieswear_Sweater',
                                        'Divided_Divided_Sweater',
                                        'Menswear_Menswear_Sweater'], title="MothlySales / TTL sales")
fig.show()

idxgrp_idx_prdtyp,ym_date,Baby/Children_Baby Sizes 50-98_Accessories set,Baby/Children_Baby Sizes 50-98_Baby Bib,Baby/Children_Baby Sizes 50-98_Ballerinas,Baby/Children_Baby Sizes 50-98_Beanie,Baby/Children_Baby Sizes 50-98_Belt,Baby/Children_Baby Sizes 50-98_Blanket,Baby/Children_Baby Sizes 50-98_Blouse,Baby/Children_Baby Sizes 50-98_Bodysuit,Baby/Children_Baby Sizes 50-98_Bootie,...,Sport_Sport_Sweater,Sport_Sport_Swimsuit,Sport_Sport_Swimwear bottom,Sport_Sport_T-shirt,Sport_Sport_Top,Sport_Sport_Trousers,Sport_Sport_Underwear bottom,Sport_Sport_Unknown,Sport_Sport_Vest top,Sport_Sport_Waterbottle
0,2018-09-01,0.0,0.0,5.240667,0.0,8.152840,0.0,0.645293,3.717630,4.526242,...,3.237728,1.303717,0.975382,2.228232,0.511049,2.207181,0.750946,0.0,1.539181,6.212523
1,2018-10-01,0.0,0.0,16.523669,0.0,10.196159,0.0,8.869282,8.478238,16.014048,...,8.943889,3.117566,2.715924,5.081075,1.220311,5.699320,2.444808,0.0,3.436732,13.385549
2,2018-11-01,0.0,0.0,9.625004,0.0,61.258684,100.0,3.229699,8.245849,15.899021,...,10.566956,4.582184,1.247495,3.834712,1.315231,5.481981,0.000000,0.0,3.376492,11.320738
3,2018-12-01,0.0,0.0,11.722761,0.0,10.196159,0.0,1.181781,6.310974,11.205342,...,5.987414,1.169114,1.827279,3.011064,1.525507,4.798074,0.000000,0.0,2.505976,12.906202
4,2019-01-01,0.0,0.0,1.819492,0.0,0.000000,0.0,4.746515,7.124582,9.005520,...,8.877938,5.020229,1.603965,5.860243,2.364620,6.538640,12.310617,0.0,4.297864,26.887769


In [16]:
display(dfgrp2.corr()[['Ladieswear_Ladieswear_Jacket']].reset_index().sort_values(by='Ladieswear_Ladieswear_Jacket').head(10))

fig = px.line(dfgrp2, x='ym_date', y=['Ladieswear_Ladieswear_Shorts',
                                        'Divided_Divided_Shorts',
                                        'Menswear_Menswear_Shorts',
                                        'Ladieswear_Lingeries/Tights_Swimwear bottom',
                                        'Ladieswear_Lingeries/Tights_Swimsuit',
                                        'Menswear_Menswear_Swimwear bottom'], title="MothlySales / TTL sales")
fig.show()

idxgrp_idx_prdtyp,idxgrp_idx_prdtyp,Ladieswear_Ladieswear_Jacket
380,Ladieswear_Lingeries/Tights_Bikini top,-0.787629
397,Ladieswear_Lingeries/Tights_Swimwear bottom,-0.777839
396,Ladieswear_Lingeries/Tights_Swimsuit,-0.746620
242,Divided_Divided_Shorts,-0.725224
375,Ladieswear_Ladieswear_Vest top,-0.719227
312,Ladieswear_Ladies Accessories_Wedge,-0.717228
487,Sport_Sport_Shorts,-0.713245
358,Ladieswear_Ladieswear_Shorts,-0.712949
306,Ladieswear_Ladies Accessories_Sunglasses,-0.712419
253,Divided_Divided_T-shirt,-0.710662


In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

feat_cols = [col for col in dfgrp2.columns if col != 'ym_date']

df_pca = StandardScaler().fit_transform(dfgrp2[feat_cols])
model_pca = PCA(n_components=5)
model_pca.fit(df_pca)
feature = model_pca.transform(df_pca)

df_eigen = model_pca.components_.T
df_eigen = pd.DataFrame(df_eigen,
                        index=None,
                        columns=['PC1','PC2','PC3','PC4','PC5'])
df_eigen['idxgrp_idx_prdtyp'] = feat_cols
df_eigen = pd.merge(
    df_eigen,
    dfgrp2.corr()[['Ladieswear_Ladieswear_Jacket']].reset_index().rename(columns={'Ladieswear_Ladieswear_Jacket': 'autumn_sales_indicator'}),
    on='idxgrp_idx_prdtyp',
    how='left'
)
px.scatter(df_eigen, x='PC1', y='PC2', hover_name='idxgrp_idx_prdtyp', color='autumn_sales_indicator')

In [18]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components=4, covariance_type='full')
gmm.fit(df_eigen[['PC1','PC2','PC3']])
df_eigen['product_seasonal_type'] = gmm.predict(df_eigen[['PC1','PC2','PC3']])
df_eigen['prob_cluster1'] = gmm.predict_proba(df_eigen[['PC1','PC2','PC3']])[:,0]
df_eigen['prob_cluster2'] = gmm.predict_proba(df_eigen[['PC1','PC2','PC3']])[:,1]
df_eigen['prob_cluster3'] = gmm.predict_proba(df_eigen[['PC1','PC2','PC3']])[:,2]
df_eigen['prob_cluster4'] = gmm.predict_proba(df_eigen[['PC1','PC2','PC3']])[:,3]
px.scatter(df_eigen, x='PC1', y='PC2', hover_name='idxgrp_idx_prdtyp', color='product_seasonal_type')

In [19]:
fig = px.line(dfgrp2, x='ym_date', y=['Ladieswear_Ladies Accessories_Hair string',
                                        'Divided_Divided_Belt',
                                        'Ladieswear_Lingeries/Tights_Unknown'], title="Sales transition")
fig.show()

In [ ]:
df = pd.merge(
    df,
    df_eigen[['idxgrp_idx_prdtyp','autumn_sales_indicator', 'product_seasonal_type', 'prob_cluster1', 'prob_cluster2', 'prob_cluster3', 'prob_cluster4']],
    on='idxgrp_idx_prdtyp',
    how='left'
)
del dfgrp1, dfgrp2
df.head()